In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision

from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import save_image
from BVAA import VAE

In [2]:
batch_size = 64

mnist_dataset = datasets.MNIST('../../data', 
                   train=True, 
                   download=True, 
                   transform=transforms.Compose([
                       transforms.Resize(32),
                       transforms.ToTensor()
                   ]))

mnist_dataset_test = datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.Resize(32),
                       transforms.ToTensor()
                   ]))

train_loader = torch.utils.data.DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(mnist_dataset_test, batch_size=batch_size, shuffle=True)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
test_batch_size = 1000
batch_size = 64
epochs = 2
lr = 0.1
gamma = 0.7
seed = 1
no_cuda = False
log_interval = 1000
save_model = True
use_cuda = True
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [9]:
model = VAE([1, 32, 32], 12)
model_old = VAE([1, 32, 32], 12)

In [10]:
model.load_state_dict(torch.load('./model/betaVAE.pt'))
model_old.load_state_dict(torch.load('./model/Siamesev1/G/G-200.pt'))

<All keys matched successfully>

In [11]:
from BVAA import Net
model_classification = Net().to(device)
optimizer = optim.Adadelta(model_classification.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [12]:
from BVAA import train, test
for epoch in range(1, epochs + 1):
    train(model_classification, device, train_loader, optimizer, epoch)
    test(model_classification, model, device, test_loader)
    test(model_classification, model_old, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294188


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
i = 34
recon_img,_,_ = model(torch.FloatTensor(example_data[i]).unsqueeze(0).to(device))
recon_img_old,_,_ = model_old(torch.FloatTensor(example_data[i]).unsqueeze(0).to(device))
plt.subplot(1,2,1)
plt.imshow(recon_img[0][0].detach().cpu().numpy(), cmap='gray', interpolation='none')
plt.title("Ground Truth: {} -> {}".format(example_targets[i], torch.argmax(model_classification(recon_img)).item()))
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(recon_img_old[0][0].detach().cpu().numpy(), cmap='gray', interpolation='none')
plt.title("Ground Truth: {} -> {}".format(example_targets[i], torch.argmax(model_classification(recon_img_old)).item()))
plt.axis('off')